In [1]:
import telegram, requests, pydub, urllib.parse, pandas ,program, os
from telegram.ext import Updater, MessageHandler, Filters, CommandHandler, CallbackQueryHandler
from telegram import InlineKeyboardButton, InlineKeyboardMarkup, InputMediaPhoto
from os import path
from pydub import AudioSegment
from bs4 import BeautifulSoup

class TelegramBot:

    def __init__(self):
        self.__token='1854272659:AAHvLiFo6ecRABj0Kb7Wk6NEkCeqnO9RaMQ'
        self.__bot = telegram.Bot(token=self.__token)
        self.__URL = ('https://api.telegram.org/bot{}/getUpdates').format(self.__token)
        self.__response = requests.get(self.__URL).json()
        self.__result = self.__response['result']
        self.__last_update_id = 0
        self.__updater = Updater(self.__token, use_context=True)
        
        result=self.__result
        print("입력된 채팅수 : {}".format(len(result)))
        
        last_update_id=self.__last_update_id
        if len(self.__result) > 0:
            last_update_id = result[len(result)-1]['update_id']
            
        print("마지막에 입력된 채팅 ID : {}".format(last_update_id))
        
        if last_update_id != 0:
            URL = ('https://api.telegram.org/bot{}/getUpdates').format(self.__token)
            response = requests.get("{}?offset={}".format(URL,last_update_id+1)).json()
        
        self.__cluster_group=[]
        for i in range(0,10):
            self.__cluster_group.append(pandas.read_csv("cluster"+str(i)+".csv"))
        self.__cluster_label_list=[]
        for i in range(len(self.__cluster_group)):
            self.__cluster_label_list.append(self.__cluster_group[i]['Label'][0])
        
        self.__comment_list=[]
        comment=open('comment.txt',encoding='UTF8')
        for data in comment.readlines():
            self.__comment_list.append(str(data)[:-1])
            
        self.music = program.RecomendMusic('Dnn.h5')
        
        print('start telegram chat bot')
        
        
    def __del__(self):
        print("클래스 종료")
        
        
    def stop_command(self, update, context):
        print("종료됬습니다.")
        self.__updater.stop()
        print("종료됬습니다.")
        
        
    def get_voice(self, update, context):
        bot = self.__bot
        file = bot.getFile(update.message.voice.file_id)
        print ("file_id: " + str(update.message.voice.file_id))
        file_id = str(update.message.voice.file_id)
        file.download("music/"+file_id+'.ogg')
        ogg_version = AudioSegment.from_ogg("music/"+file_id+'.ogg')
        ogg_version.export("music/"+file_id+'.wav', format="wav")
        # 버튼 생성
        context.bot.send_message(
        chat_id=update.message.chat_id,
        text="<b>잠시만 기다려 주시겠어요?</b>\n___노래목록__생성중___",
        parse_mode="HTML")
        
        feature_df, mfccs = self.music.feature_extraction("music/"+file_id+'.wav')
        dis_df = self.music.dtw_distance(mfccs)
        input_data = self.music.make_input_data(feature_df, dis_df)
        df=self.music.recomending(input_data)
            
        media_list=[]
        buttons_list=[]
            
        for data in df['Title']:
            artist=str(data).split('_')[0]
            html = "https://www.melon.com/search/total/index.htm?q="+ urllib.parse.quote(artist)+'&section=&linkOrText=T&ipath=srch_form'
            header01 = {'User-Agent':'Mozilla/5.0'}
            req=requests.get(html, headers=header01)
            soup = BeautifulSoup(req.text, 'html.parser')
            try:
                photo_url=str(soup.find_all('img')[2])
                if photo_url.find('.jpg') == -1:
                    photo_url = open('img.jpg', 'rb')
                else:
                    photo_url=photo_url.split('src="')[1]
                    photo_url=photo_url.split('.jpg')[0]+".jpg"
            except:
                photo_url = open('img.jpg', 'rb')
            media_list.append(InputMediaPhoto(media=photo_url,caption=str(data)))
            buttons_list.append([InlineKeyboardButton(str(data),"https://www.youtube.com/results?search_query="+urllib.parse.quote(str(data).replace(" ","+")) )])
            
        self.__bot.send_media_group(chat_id=update.message.chat_id,
                                        media=media_list)
            
        buttons = buttons_list
            
        reply_markup = InlineKeyboardMarkup(buttons)
        Prediction = 0
        for i in range(len(self.__cluster_label_list)):
            if df['Label'].unique()[0] == self.__cluster_label_list[i]:
                Prediction = i
            
        context.bot.send_message(
            text ='<b>"'+df['Label'].unique()[0]+'"</b>\n'+ self.__comment_list[Prediction],
            parse_mode="HTML",
            chat_id=update.message.chat_id,
            reply_markup=reply_markup
            )
        os.remove("music/"+file_id+'.ogg')
        os.remove("music/"+file_id+'.wav')
    
    
    def music_recommend_buttons(self, update, context):
        music_recommend_buttons_list=[]
        for i in range(0,10):
            music_recommend_buttons_list.append([InlineKeyboardButton( self.__cluster_group[i]['Label'][0] ,callback_data=i)])
        buttons = music_recommend_buttons_list

        reply_markup = InlineKeyboardMarkup( buttons )

        context.bot.send_message(
            chat_id=update.message.chat_id
            , text="<b>원하시는 취향을 선택해주세요</b>"
            ,parse_mode="HTML",
             reply_markup=reply_markup)
        

    def callback_get(self, update, context):
        input_data=update.callback_query.data
        if int(input_data) >=0 and int(input_data) <=9:
            df_cluster=self.__cluster_group[int(input_data)]
            recommand_song_list=df_cluster.sample(5)['Title']
            
            buttons_list = []
            for data in recommand_song_list:
                buttons_list.append([InlineKeyboardButton(str(data),"https://www.youtube.com/results?search_query="+urllib.parse.quote(str(data).replace(" ","+")) )])
            
            buttons = buttons_list
            
            reply_markup = InlineKeyboardMarkup(buttons)
            
            context.bot.send_message(
                text='<b>"'+self.__cluster_group[int(input_data)]['Label'][0]+'"</b>\n'+self.__comment_list[int(input_data)],
                parse_mode="HTML",
                chat_id=update.callback_query.message.chat_id,
                reply_markup=reply_markup
                )
            
            
    def runUpdate(self):
        self.index()
    
    
    def index(self):
        updater = self.__updater
        
        voice_handler = MessageHandler(Filters.voice, self.get_voice)
        updater.dispatcher.add_handler(voice_handler)
        
        updater.dispatcher.add_handler(CallbackQueryHandler(self.callback_get))
        
        music_handler = CommandHandler('music', self.music_recommend_buttons)
        updater.dispatcher.add_handler(music_handler)
        
        stop_handler = CommandHandler('stop', self.stop_command)
        updater.dispatcher.add_handler(stop_handler)
        
        updater.start_polling(poll_interval=3,drop_pending_updates=True)

In [2]:
telegramBot=TelegramBot()
telegramBot.runUpdate()
del telegramBot

입력된 채팅수 : 0
마지막에 입력된 채팅 ID : 0
start telegram chat bot
file_id: AwACAgUAAxkBAAIGJWC93NxylqxoQd_AXsXyDgnnIohtAALJAgACCfrxVQWhwp9bjWDGHwQ


C:\Users\020\anaconda3\lib\site-packages\librosa\filters.py:238: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  warnings.warn(


Prediction: 1
file_id: AwACAgUAAxkBAAIGLWC93SNAP6yXV7GFRvF_EG9_RpbvAAJRAwACKNnpVVdxqeyWUaN6HwQ


C:\Users\020\anaconda3\lib\site-packages\librosa\filters.py:238: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  warnings.warn(


Prediction: 3
file_id: AwACAgUAAxkBAAIGNWC93Vc4NWvm4qJm1CyiD1tk6urXAAJSAwACKNnpVfBU_ik4FotfHwQ


...

Prediction: 3
file_id: AwACAgUAAxkBAAIGcWC95zUfdZkUsnaYliu_pWG_TQdmAAIXAgADJ_FV1URyQQdHIFcfBA


C:\Users\020\anaconda3\lib\site-packages\librosa\filters.py:238: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  warnings.warn(


Prediction: 3
file_id: AwACAgUAAxkBAAIGeWC95_DGy0ehWgGFMm6oDpPF8liHAAIYAgADJ_FViezDdyFWgs0fBA


C:\Users\020\anaconda3\lib\site-packages\librosa\filters.py:238: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  warnings.warn(


Prediction: 1
